In [38]:
import os
import glob
import sys

reduce_factor=2
new_size=400
delay=20

In [39]:
files_list = sorted([g for j,g in enumerate(glob.glob('*.png')) if int(g.split('-')[1].split('.')[0])<500],
                    key=lambda rw: int(rw.split('-')[1].split('.')[0]))[::reduce_factor]
print(files_list[0:5])

['dwaveFrame-0000.png', 'dwaveFrame-0002.png', 'dwaveFrame-0004.png', 'dwaveFrame-0006.png', 'dwaveFrame-0008.png']


In [40]:

convert_str = 'convert -delay {} -loop 0 -resize {}x {} animated_dwf.gif'.format(delay, new_size, 
    ' '.join(files_list)) 

#print(convert_str)
os.system(convert_str)

0